## Creating Datasets from the Dengue Hierarchy and interactome.
- The goal is to generate datasets in various formats based on the assemblies in a hierarchical model.
- This involves:
    - filtering the assemblies on assembly names, min size, and max size
    - filtering the data by columns and by row values
    - changing column names
    - generating/adding experiment, assembly, and content descriptions
    - cleaning the data, such as non-numeric values
    - limiting the precision of numeric values
    - optionaly saving the datasets to the database
    - optionally adding interaction data
    - optionally adding information from other sources, such as genecards
- The Dengue data is on the STRING+diffusion based interactome
- We can also get the data directly from the dengue_with_uniprot.csv
- Laura Martin-Sancho is most interested in the assemblies listed in the interesting_dengue_communities.xlsx spreadsheet

In [12]:
import sys
import os

# Add the parent directory of the current script to the Python path
cwd = os.getcwd()
dirname = os.path.dirname(cwd)
print(cwd)
print(dirname)
sys.path.append(dirname)

print(sys.path)

from models.analysis_plan import AnalysisPlan
from services.analysisrunner import AnalysisRunner
from models.review_plan import ReviewPlan
from services.reviewrunner import ReviewRunner
from app.sqlite_database import SqliteDatabase
from app.config import load_database_config

# Load the db connection details
# db_type, uri, user, password = load_database_config(path='~/ae_config/test_config.ini')
# self.db = Database(uri, db_type, user, password)

_, database_uri, _, _ = load_database_config()
db = SqliteDatabase(database_uri)

/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks
/Users/idekeradmin/Dropbox/GitHub/agent_evaluation
['/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks', '/opt/anaconda3/lib/python311.zip', '/opt/anaconda3/lib/python3.11', '/opt/anaconda3/lib/python3.11/lib-dynload', '', '/Users/idekeradmin/.local/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages/aeosa', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation']


In [13]:


dengue_column_name_mapping ={}

## Get the assemblies of interest

In [14]:
import pandas as pd
cwd = os.getcwd()
dirname = os.path.dirname(cwd)
assembly_spreadsheet_filename = "interesting_dengue_communities.xlsx"
top_20_assembly_names_df = pd.read_excel(os.path.join(dirname, 
                                                      "files", 
                                                      assembly_spreadsheet_filename),
                                                      sheet_name=0)
top_10_assembly_names_df = pd.read_excel(os.path.join(dirname, 
                                                      "files", 
                                                      assembly_spreadsheet_filename),
                                                      sheet_name=1)

## Get the model and the interactome from NDEx in CX2

In [15]:
from models.hierarchy import Hierarchy
import json
import ndex2 
from ndex2.cx2 import RawCX2NetworkFactory

# Create NDEx2 python client
client = ndex2.client.Ndex2()

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Download BioGRID: Protein-Protein Interactions (SARS-CoV) from NDEx
# https://www.ndexbio.org/viewer/networks/669f30a3-cee6-11ea-aaef-0ac135e8bacf
# client_resp = client.get_network_as_cx2_stream('669f30a3-cee6-11ea-aaef-0ac135e8bacf')

# Dengue string interactome network c223d6db-b0e2-11ee-8a13-005056ae23aa
client_resp = client.get_network_as_cx2_stream('c223d6db-b0e2-11ee-8a13-005056ae23aa')

# Convert downloaded interactome network to CX2Network object
interactome = factory.get_cx2network(json.loads(client_resp.content))

# Dengue hierarchy
# https://www.ndexbio.org/viewer/networks/59bbb9f1-e029-11ee-9621-005056ae23aa
client_resp = client.get_network_as_cx2_stream('59bbb9f1-e029-11ee-9621-005056ae23aa')

# Convert downloaded interactome network to CX2Network object
hierarchy = factory.get_cx2network(json.loads(client_resp.content))

# Display information about the hierarchy network and output 1st 100 characters of CX2
print('Name: ' + hierarchy.get_name())
print('Number of nodes: ' + str(len(hierarchy.get_nodes())))
print('Number of nodes: ' + str(len(hierarchy.get_edges())))

# Display information about the interactome network 
print('Name: ' + interactome.get_name())
print('Number of nodes: ' + str(len(interactome.get_nodes())))
print('Number of nodes: ' + str(len(interactome.get_edges())))

# this is brief so that it will help keep the context small for fast operation.
brief_dengue_dataset_description = """
The dataset includes the following for genes/proteins: 

"binds": Dengue virus proteins bound to the human protein
"knockdown_inhbits": 1 = siRNA knockdown inhibits dengue virus infection, 2 = no effect

rna and protein expression changes following dengue virus infection:
"protein_logFC_24h"
"protein_log2FC_48h"
"rna_logFC_24h"
"rna_log2FC_48h"
"""
      
hierarchy.add_network_attribute("experiment_description", brief_dengue_dataset_description)


Name: Dengue model - hidef string 12.0 0.7 (GPT-4 annotated) - L2R
Number of nodes: 203
Number of nodes: 249
Name: dengue string 12.0 0.7
Number of nodes: 1375
Number of nodes: 2792


## Preview the data columns

In [16]:
# csv_path = os.path.join(dirname, "files", "dengue_with_uniprot.csv")
# data = pd.read_csv(csv_path)

excel_path = os.path.join(dirname, "files", "dengue_with_uniprot_full.xlsx")
data = pd.read_excel(excel_path)

# data.columns.to_list()
column_dict = {}
for column in data.columns:
    column_dict[column] = column
column_dict

{'GeneID': 'GeneID',
 'UniprotID': 'UniprotID',
 'GeneSymbol': 'GeneSymbol',
 'siRNA_GeneSymbol': 'siRNA_GeneSymbol',
 'DV3_24h-Mock_24h': 'DV3_24h-Mock_24h',
 'DV3_48h-Mock_48h': 'DV3_48h-Mock_48h',
 'siRNA_Screen_Average_Zscore': 'siRNA_Screen_Average_Zscore',
 'log2FC': 'log2FC',
 'Condition': 'Condition',
 'GeneSymbol_48hpi': 'GeneSymbol_48hpi',
 'log2FC_48hpi': 'log2FC_48hpi',
 'Condition_48hpi': 'Condition_48hpi',
 'dengue_protein_list': 'dengue_protein_list',
 'dengue_MiST_list': 'dengue_MiST_list',
 'PPI_GeneSymbol': 'PPI_GeneSymbol',
 'viral_interaction': 'viral_interaction',
 'has_siRNA': 'has_siRNA',
 'has_protein_24hr': 'has_protein_24hr',
 'has_protein_48hr': 'has_protein_48hr',
 'has_rnaSeq_24hr': 'has_rnaSeq_24hr',
 'has_rnaSeq_48hr': 'has_rnaSeq_48hr',
 'UniProtID': 'UniProtID',
 'HGNC': 'HGNC'}

## Make the Hierarchy object and annotate it with data
- Select the columns and optionally rename them
- Only annotate those selected by the optional filter, based on name and size range
- Optionally reduce the precision of floats


In [17]:
import os

dengue_hierarchy = Hierarchy(hierarchy, interactome)

# csv_path = os.path.join(dirname, "files", "dengue_with_uniprot.csv")
excel_path = os.path.join(dirname, "files", "dengue_with_uniprot_full.xlsx")

assembly_list = top_10_assembly_names_df["Community"].to_list()

data_columns = columns={'GeneID': 'GeneID',
                        # 'UniprotID': 'UniprotID',
                        # 'GeneSymbol': 'GeneSymbol',
                        # 'siRNA_GeneSymbol': 'siRNA_GeneSymbol',
                        'DV3_24h-Mock_24h': 'rna_log2FC-24h',
                        'DV3_48h-Mock_48h': 'rna_log2FC_48h',
                        # 'siRNA_Screen_Average_Zscore': 'siRNA_Screen_Average_Zscore',
                        'log2FC': 'protein_log2FC_24h',
                        #'Condition': 'Condition',
                        # 'GeneSymbol_48hpi': 'GeneSymbol_48hpi',
                        'log2FC_48hpi': 'protein_log2FC_48h',
                        # 'Condition_48hpi': 'Condition_48hpi',
                        'dengue_protein_list': 'binds',
                        # 'dengue_MiST_list': 'dengue_MiST_list',
                        # 'PPI_GeneSymbol': 'PPI_GeneSymbol',
                        # 'viral_interaction': 'viral_interaction',
                        'has_siRNA': 'knockdown_inhibits',
                        # 'has_protein_24hr': 'has_protein_24hr',
                        # 'has_protein_48hr': 'has_protein_48hr',
                        # 'has_rnaSeq_24hr': 'has_rnaSeq_24hr',
                        # 'has_rnaSeq_48hr': 'has_rnaSeq_48hr',
                        # 'UniProtID': 'UniProtID',
                        'HGNC': 'GeneSymbol'}

print(data_columns)
# add_data_from_file(self, file_path, key_column='name', columns=None, filter=None, sheet_name=0, delimiter=None):
dengue_assemblies = dengue_hierarchy.add_data_from_file(excel_path,
                                                 key_column="HGNC",
                                                 filter={"names": assembly_list},
                                                 columns=data_columns,
                                                 decimal_places=2)

dengue_assemblies[0]

{'GeneID': 'GeneID', 'DV3_24h-Mock_24h': 'rna_log2FC-24h', 'DV3_48h-Mock_48h': 'rna_log2FC_48h', 'log2FC': 'protein_log2FC_24h', 'log2FC_48hpi': 'protein_log2FC_48h', 'dengue_protein_list': 'binds', 'has_siRNA': 'knockdown_inhibits', 'HGNC': 'GeneSymbol'}
mapped key column GeneSymbol is not in data row {'GeneID': 440519, 'protein_log2FC_48h': -2.47, 'knockdown_inhibits': 0, 'GeneSymbol': 'ZNF724'}
mapped key column GeneSymbol is not in data row {'GeneID': 440519, 'protein_log2FC_48h': -2.47, 'knockdown_inhibits': 0, 'GeneSymbol': 'ZNF724'}
mapped key column GeneSymbol is not in data row {'GeneID': 440519, 'protein_log2FC_48h': -2.47, 'knockdown_inhibits': 0, 'GeneSymbol': 'ZNF724'}
mapped key column GeneSymbol is not in data row {'GeneID': 440519, 'protein_log2FC_48h': -2.47, 'knockdown_inhibits': 0, 'GeneSymbol': 'ZNF724'}
mapped key column GeneSymbol is not in data row {'GeneID': 440519, 'protein_log2FC_48h': -2.47, 'knockdown_inhibits': 0, 'GeneSymbol': 'ZNF724'}
mapped key column G

{'id': 5971062,
 'v': {'CD_Labeled': True,
  'CD_AnnotatedAlgorithm': 'Annotated by gProfiler [Docker: coleslawndex/cdgprofilergenestoterm:0.3.0] {{--organism=hsapiens, --maxpval=0.00001, --minoverlap=0.05, --maxgenelistsize=5000}} via CyCommunityDetection Cytoscape App (1.10.0-SNAPSHOT)',
  'name': 'C4493683',
  'CommunityDetectionTally::viral_interaction': 1,
  'CD_AnnotatedMembers_Pvalue': 7.71565013380798e-21,
  'CD_AnnotatedMembers_Size': 11,
  'CD_AnnotatedMembers_Overlap': 0.091,
  'CommunityDetectionTally::has_protein_24hr': 0,
  'CD_MemberList_LogSize': 5.459,
  'CommunityDetectionTally::has_protein_48hr': 2,
  'CommunityDetectionTally::has_siRNA': 19,
  'CD_MemberList': 'BMP2 CAV1 CLDN5 COL15A1 COL6A2 COL6A5 DCC DMP1 ESPNL F13A1 FLNA FSTL1 IGFBP7 ITGA11 ITGA2 ITGA9 ITGAD ITGB3 ITGB8 LAMA3 LAMC2 MSN MSX1 NBL1 NCF1 NEO1 OSR2 PF4V1 PHEX PLS1 PTPN13 RND3 SAT1 SELP SERPINH1 SMAD6 SNX17 SPARC STAB1 STOM SYNPO2 UNC5C VCAM1 VWF',
  'LLM Name': 'Extracellular Matrix Organization and C

## Generate Datasets for a list of assemblies
- optionally selecting columns
- get a list of their ids to use elsewhere

In [18]:
from models.hierarchy import dataset_from_assembly

dataset_ids = []

for assembly in dengue_assemblies:
    dataset = dataset_from_assembly(db, assembly, experiment_description=brief_dengue_dataset_description)
    print(dataset.name)
    dataset_ids.append(dataset.object_id)
 

"Extracellular Matrix Organization and Cell Adhesion in Response to Viral Infection"
"Extracellular Matrix Organization and Cell Adhesion in Response to Viral Infection"
"Urea cycle and redox homeostasis"
"DNA Repair and Maintenance in Response to Viral Infection"
"RAS Pathway Modulation and Apoptosis Regulation"
"Oxidative Stress Response and Protein Quality Control in Neurodegeneration"
"RNA Metabolism and Viral Defense Mechanism"
"Kynurenine Pathway Modulation and Immune Response Regulation"
"RNA Metabolism and Modification in Viral Infection"
"Endothelial Barrier Function and Viral Entry Modulation"


In [19]:
#```python
def list_assemblies(db, hierarchy, filter=None):
    for assembly in hierarchy.get_assemblies(filter=filter):
        #print(json.dumps(assembly, indent=4))
        print(assembly["v"].get("name"))
        print(f'--LLM-> {assembly["v"].get("LLM Name")}')
        print(f'--ENR-> {assembly["v"].get("CD_CommunityName")}')
        print(assembly['v'].get('data'))
        #print(dataset_from_assembly(db, assembly))
        print('___________________________________________________________________')

list_assemblies(db, dengue_hierarchy, filter={"names": top_10_assembly_names_df["Community"].to_list(),
                                              "columns": ["binds", 
                                                          "knockdown_inhbits", 
                                                          "protein_logFC_24h", "protein_log2FC_48h",
                                                          "rna_logFC_24h", "rna_log2FC_48h"]})
#```

C4493683
--LLM-> Extracellular Matrix Organization and Cell Adhesion in Response to Viral Infection (0.85)
--ENR-> ECM-receptor interaction
{"STOM": {"GeneID": 2040, "rna_log2FC_48h": 1.88, "protein_log2FC_48h": 2.08, "binds": "DENV2 16681 NS4A,ZIKVfp NS4A,ZIKVug NS4A", "knockdown_inhibits": 0, "GeneSymbol": "STOM"}, "SERPINH1": {"GeneID": 871, "rna_log2FC_48h": 1.07, "knockdown_inhibits": 0, "GeneSymbol": "SERPINH1"}, "CLDN5": {"GeneID": 7122, "knockdown_inhibits": 1, "GeneSymbol": "CLDN5"}, "COL15A1": {"GeneID": 1306, "knockdown_inhibits": 1, "GeneSymbol": "COL15A1"}, "DCC": {"GeneID": 1630, "knockdown_inhibits": 1, "GeneSymbol": "DCC"}, "COL6A5": {"GeneID": 256076, "knockdown_inhibits": 1, "GeneSymbol": "COL6A5"}, "IGFBP7": {"GeneID": 3490, "knockdown_inhibits": 1, "GeneSymbol": "IGFBP7"}, "ITGA11": {"GeneID": 22801, "knockdown_inhibits": 1, "GeneSymbol": "ITGA11"}, "ITGAD": {"GeneID": 3681, "knockdown_inhibits": 1, "GeneSymbol": "ITGAD"}, "LAMC2": {"GeneID": 3918, "knockdown_inhibi

```
from models.dataset import Dataset

c4493827 = Dataset.load(db, "dataset_ac5bf71e-9e92-4004-8c49-3d62f538044a")

print(c4493827.data)
```

```
You are a brilliant graduate student in molecular biology. Your work is reviewed by your world-renowned advisor, who is very demanding. You will need to give your very best effort in this project.
```
----------

```
<task>
Based on the experiment description, analyze the dataset and develop a mechanistic, causal hypothesis for the processes that led to the observed data. The proteins/genes in the dataset are hypothesized to be known to interact; use your knowledge of these proteins and their interactions to develop chains of events that connect experimental perturbations with molecular and phenotypic observations. The hypothesis should make specific predictions that could be experimentally validated. 
</task>

<biological_context>
The hypothesis should support the higher-level goal of developing drug therapies for the disease.
</biological_context>

<detailed_instructions>
1. Review the data to ensure that you understand the meaning of each observation. In your hypothesis, be sure that you correctly use the data and do not hallucinate any observations.

2. Review your knowledge of the functions of these proteins and the known interactions between them.

3. Based on the data and your knowledge, construct the hypothesis that you think best meets these criteria:
- Plausible
- Non-trivial
- Supports the higher-level goal of drug development
- Novel
- Actionable: is cost-effective in both time and money

4. Your lab has limited resources in both reagents, equipment, and your time. Remember, your time is precious. You must use it well if you are to get your doctorate. If you do not think that there is any hypothesis that is worth following up with a validation experiment, say so.

5. Output your hypothesis as follows:

## Knowledge Graph:
concise knowledge graph of as a list of the causal relationships between proteins, complexes, events, disease states, etc. "Therefore:" indicates hypothesized relationships. For example:

(A binds B) inactivates B
B performs (phosphorylation C)
(phosphorylation C) increases (active C)
Therefore: A decreases (active C)


## Hypothesis:
short descriptive paragraph. 

</detailed_instructions>

<experiment_description>
{experiment_description}
</experiment_description>

<dataset>
{data}
</data>
```